In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

## Data Acquisition
These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

* Read the case, department, and source data into their own spark dataframes.

In [2]:
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [3]:
# Read in CSV file 
case = (spark.read.csv("case.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [4]:
# Read in CSV file 
dept = (spark.read.csv("dept.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [5]:
source.show(5) ## <-- previewing source

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [6]:
case.show(3, False, True) ## <-- previewing case

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

In [7]:
dept.show(5) ## <-- previewing departments

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



Let's see how writing to the local disk works in spark:

* Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
* Inspect your folder structure. What do you notice?

In [8]:
## writing my Json files since I already downloaded the csv's from google classroom

case.write.json('data/case_json', mode = 'overwrite') 

source.write.json('data/source_json', mode = 'overwrite')

dept.write.json('data/dept_json', mode = 'overwrite')

In my local computer spark-exercises folder I notice that this code has made three json files and organized them into a subfolder named 'data'

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [9]:
## inspecting our spark dataframe date types using dtypes

case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [10]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [11]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

Things to do:

Rename Columns:
* 'SLA_due_date -> case_due_date

Correct Data Types:
* case_closed and case_late to boolean
* council_district as a string
* case_opened_date, case_closed_date and case_due_date to datetime format

Data Transformation:
* request_address: trim and lowercase
* format council district with leading zeros
* convert the number of days a case is late to a number of weeks

New features:
* zip_code : extract from address
* case_age
* days_to_closed
* case_lifetime

Join cases data with department data:

Rename The Columns:

In [12]:
# Rename 'SLA_due_date' to 'case_due_date' using .withColumnRenamed

case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

In [13]:
case.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 case_due_date        | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



Correct Data Types:

In [14]:
# correct data types: case_closed and case_late to boolean

case.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|      YES|
+-----------+---------+
only showing top 5 rows



In [15]:
# use .withColumn to change columns from string to boolean values

case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

In [16]:
case.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [17]:
# council_district cast as string
case.select('council_district').show(4)

+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
|               3|
+----------------+
only showing top 4 rows



In [18]:
# council_district as a string instead of int
case = case.withColumn('council_district', col('council_district').cast('string'))

In [19]:
# view the column

case.select('council_district').show(4)

+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
|               3|
+----------------+
only showing top 4 rows



In [20]:
# check datatypes
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('case_due_date', 'string'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

In [21]:
# convert case_opened_date, case_closed_date and case_due_date to datetime format

case.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows



In [22]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('case_due_date', to_timestamp('case_due_date', fmt))

In [23]:
# check the three columns again

case.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



Data Transformation

In [24]:
# request_address: trim and lowercase

case.select('request_address').show(5, False)

+-------------------------------------+
|request_address                      |
+-------------------------------------+
|2315  EL PASO ST, San Antonio, 78207 |
|2215  GOLIAD RD, San Antonio, 78223  |
|102  PALFREY ST W, San Antonio, 78223|
|114  LA GARDE ST, San Antonio, 78223 |
|734  CLEARVIEW DR, San Antonio, 78228|
+-------------------------------------+
only showing top 5 rows



In [25]:
case = case.withColumn('request_address', trim(lower(case.request_address)))

case.select('request_address').show(5, False)

+-------------------------------------+
|request_address                      |
+-------------------------------------+
|2315  el paso st, san antonio, 78207 |
|2215  goliad rd, san antonio, 78223  |
|102  palfrey st w, san antonio, 78223|
|114  la garde st, san antonio, 78223 |
|734  clearview dr, san antonio, 78228|
+-------------------------------------+
only showing top 5 rows



In [26]:
# convert the number of days a case is late to a number of weeks

case = case.withColumn('num_weeks_late', expr('num_days_late/7'))

case.select("num_days_late", "num_weeks_late").show(5)

+-------------------+--------------------+
|      num_days_late|      num_weeks_late|
+-------------------+--------------------+
| -998.5087616000001|        -142.6441088|
|-2.0126041669999997|-0.28751488099999994|
|       -3.022337963|-0.43176256614285713|
|       -15.01148148| -2.1444973542857144|
|0.37216435200000003|         0.053166336|
+-------------------+--------------------+
only showing top 5 rows



In [27]:
# use format_string function to pad zeros for council_district

case = case.withColumn('council_district', format_string('%03d', col('council_district').cast('int')))

In [28]:
case.select('council_district').show(5)

+----------------+
|council_district|
+----------------+
|             005|
|             003|
|             003|
|             003|
|             007|
+----------------+
only showing top 5 rows



New Features: 

In [29]:
# create a new column for zipcode:

case = case.withColumn('zipcode', regexp_extract('request_address', r"(\d+$)", 1))

case.select('zipcode').show(5)

+-------+
|zipcode|
+-------+
|  78207|
|  78223|
|  78223|
|  78223|
|  78228|
+-------+
only showing top 5 rows



In [30]:
case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode

case_age: How old the case is; the difference in days between when the case was opened and the current day

days_to_closed: The number of days between when the case was opened and when it was closed

case_lifetime: Number of days between when the case was opened and when it was closed, if the case is still open, the number of days since the case was opened

In [31]:
#create three new columns 'case_age', 'days_to_closed', 'case_lifetime'

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [32]:
case.select(expr('CASE WHEN case_closed == False THEN case_age ELSE days_to_closed END').alias('case_lifetime')).show(3)

+-------------+
|case_lifetime|
+-------------+
|            0|
|            2|
|            1|
+-------------+
only showing top 3 rows



In [33]:
case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode

Question 3: Before joining tables working with original case:
* How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

Joining dept on case

In [34]:
## using where to filter for filed operations department as well as 
## officer stanby requests and getting the answer with a count()

(case.where(expr('dept_division == "Field Operations"'))
.where(expr('service_request_type != "Officer Standby"'))).count()

113902

In [35]:
# join the df and dept dataframe using 'dept_division' as common key
# drop columns as needed (keep standardized_dept_name)
# convert dept_subject_to_SLA to boolean

case = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [36]:
case.show(1, False, True) ## <-- previewing our new dataframe

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode              | 78207                                
 case_ag

#### Exercise Questions

Question 1: How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [37]:
## using where to filter for open cases and ordering by longest SLA days in descending order

case.where(case.case_status == 'Open').orderBy(desc(case.SLA_days)).show(1, False, True)

-RECORD 0--------------------------------------------------------
 case_id              | 1013896575                               
 case_opened_date     | 2017-09-22 08:27:00                      
 case_closed_date     | null                                     
 case_due_date        | 2021-08-11 08:30:00                      
 case_late            | false                                    
 num_days_late        | -1318.354167                             
 case_closed          | false                                    
 service_request_type | Signal Timing Modification By Engineer   
 SLA_days             | 1419.00191                               
 case_status          | Open                                     
 source_id            | CRM_Listener                             
 request_address      | 4200  harry wurzbach, san antonio, 78209 
 council_district     | 010                                      
 num_weeks_late       | -188.33630957142856                      
 zipcode  

Question 2: How many Stray Animal cases are there? 

In [38]:
## using where to filter for stray animal request types and counting them with count()

case.where(case.service_request_type == 'Stray Animal').count()

26760

Question 3: How many service requests that are assigned to the Field Operations department
* (dept_division) are not classified as "Officer Standby" request type
* (service_request_type)?

In [39]:
## using where to filter for filed operations department as well as 
## officer stanby requests and getting the answer with a count()

(case.where(expr('department == "Field Operations"'))
.where(expr('service_request_type != "Officer Standby"'))).count()

0

Question 4: Convert the council_district column to a string column

In [40]:
case.dtypes ## this was already done preparing the dataframe for the exercise

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('num_weeks_late', 'double'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int'),
 ('department', 'string'),
 ('dept_subject_to_SLA', 'boolean')]

Question 5: Extract the year from the case_closed_date column.

In [41]:
case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode              | 78207                                
 case_ag

In [42]:
## using datetime format to extract the year from case closed date column and make it a 
## column

case = case.withColumn('case_year', year(col('case_closed_date')))

case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode              | 78207                                
 case_ag

Question 6: Convert num_days_late from days to hours in new columns num_hours_late.



In [43]:
## turining days into hours by multiplying days late by 24 (hours in a day)

case = case.withColumn('num_hours_late', (col('num_days_late') * 24))

In [44]:
case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode              | 78207                                
 case_ag

Question 7: Join the case data with the source and department data.

* Answer put in raw cell because this was already done above.


Question 8: Are there any cases that do not have a request source?



In [45]:
case.where(case.source_id == 'null').count()

0

Question 9: What are the top 10 service request types in terms of number of requests?



In [52]:
(
    (case.groupby('service_request_type').count()).orderBy(desc(col('count')))
    .show(10, truncate = False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



Question 10: What are the top 10 service request types in terms of average days late?


In [59]:
(
    (case.where(case.case_late == 'YES').groupby('service_request_type')
     .agg(mean('num_days_late')))
    .orderBy(desc(col('avg(num_days_late)')))
    .show(10, truncate = False)
)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.20517608494276|
|Structure/Housing Maintenance         |190.20707698509807|
|Donation Container Enforcement        |171.09115313942615|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



Question 11: Does number of days late depend on department?


In [61]:
(
    (case.where(case.case_late == 'YES').groupby('department')
     .agg(mean('num_days_late')))
    .orderBy(desc(col('avg(num_days_late)')))
    .show(10, truncate = False)
)

+------------------------+------------------+
|department              |avg(num_days_late)|
+------------------------+------------------+
|Customer Service        |87.68385942150394 |
|DSD/Code Enforcement    |49.38428705358908 |
|Animal Care Services    |23.458633245820124|
|Parks & Recreation      |22.348910457867518|
|Trans & Cap Improvements|10.603064680316946|
|Solid Waste             |7.186821906120899 |
|Metro Health            |6.5438133155476494|
+------------------------+------------------+



Based on the different departments have a large range of average number of days late. I think we can conclude the number of days late can depend on the department

Question 12: How do number of days late depend on department and request type?


In [66]:
(
    (case.where(case.case_late == 'YES').groupby(['department', 'service_request_type'])
     .agg(mean('num_days_late')))
    .orderBy(desc(col('avg(num_days_late)')))
    .show(case.count(), False)
)



+------------------------+-----------------------------------------------+-------------------+
|department              |service_request_type                           |avg(num_days_late) |
+------------------------+-----------------------------------------------+-------------------+
|DSD/Code Enforcement    |Zoning: Recycle Yard                           |210.89201994318182 |
|DSD/Code Enforcement    |Zoning: Junk Yards                             |200.20517608494276 |
|DSD/Code Enforcement    |Structure/Housing Maintenance                  |190.20707698509807 |
|DSD/Code Enforcement    |Donation Container Enforcement                 |171.09115313942615 |
|DSD/Code Enforcement    |Storage of Used Mattress                       |163.96812829714287 |
|DSD/Code Enforcement    |Labeling for Used Mattress                     |162.43032902285717 |
|DSD/Code Enforcement    |Record Keeping of Used Mattresses              |153.99724039428568 |
|DSD/Code Enforcement    |Signage Requied for Sale

For the department and request type I would conclude we would need to do some statistical testing of subgroups and used a two tailed t test to determine significance. There are lot of requests and similar request types 